# Data science in Python

- Course GitHub repo: https://github.com/pycam/python-data-science
- Python website: https://www.python.org/ 

## Session 2.3: Biological data with BioPython


- [BioPython](#BioPython)
- [Working with sequences](#Working-with-sequences)
- [Connecting with biological databases](#Connecting-with-biological-databases)
- [Exercise 2.3.1](#Exercise-2.3.1)

## BioPython

The goal of Biopython is to make it as easy as possible to use Python for bioinformatics by creating high-quality, reusable modules and classes. Biopython features include parsers for various Bioinformatics file formats (BLAST, Clustalw, FASTA, Genbank,...), access to online services (NCBI, Expasy,...), interfaces to common and not-so-common programs (Clustalw, DSSP, MSMS...), a standard sequence class, various clustering modules, a KD tree data structure etc. and documentation as well as a tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html.

The Biopython Application Programming Interface (API) documentation: http://biopython.org/DIST/docs/api/.

## Working with sequences

We can create a sequence by defining a `Seq` object with strings. `Bio.Seq()` takes as input a string and converts in into a Seq object. We can print the sequences, individual residues, lengths and use other functions to get summary statistics. 

In [1]:
from Bio.Seq import Seq

my_dna = Seq("AGTACATGACTGGTTTAG")
print(my_dna)

AGTACATGACTGGTTTAG


The Seq class *inherets* from string, and so all string methods work with it

In [9]:
print(my_dna)
print(my_dna[10])
print(my_dna[1:5])
print(len(my_dna))
print(my_dna.count("A"))
print(my_dna+my_dna)

AGTACATGACTGGTTTAG
T
GTAC
18
5
AGTACATGACTGGTTTAGAGTACATGACTGGTTTAG


In [16]:
my_dna.complement()

TCATGTACTGACCAAATC


In [ ]:
my_dna.reverse_complement()

In [15]:
my_dna.translate()

Seq('ST*LV*', HasStopCodon(IUPACProtein(), '*'))

We can use functions from `Bio.SeqUtils` to get idea about a sequence 

In [6]:
# Calculate the molecular weight
from Bio.SeqUtils import GC, molecular_weight
print(GC(my_dna))
print(molecular_weight(my_dna))

38.888888888888886
5633.600099999999


One letter code protein sequences can be converted into three letter codes using `seq3` function

In [17]:
from Bio.SeqUtils import seq3
print(seq3(my_dna))

AlaGlyThrAlaCysAlaCysThrGlyGlyThr


### Parsing sequence file format: FASTA files

Sequence files can be parsed and read the same way we read other files. 

In [7]:
with open( "data/glpa.fa" ) as f:
    print(f.read())

>swissprot|P02724|GLPA_HUMAN Glycophorin-A;
MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAH
EVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRLIKK
SPSDVKPLPSPDTDVPLSSVEIENPETSDQ




Biopython provides specific functions to allow parsing/reading sequence files. 

In [8]:
# Reading FASTA files
from Bio import SeqIO

with open("data/glpa.fa") as f:
    for protein in SeqIO.parse(f, 'fasta'):
        print(protein.id)
        print(protein.seq)

swissprot|P02724|GLPA_HUMAN
MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAHEVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFGVMAGVIGTILLISYGIRRLIKKSPSDVKPLPSPDTDVPLSSVEIENPETSDQ


Sequence objects can be written into files using file handles with the function `SeqIO.write()`. We need to provide the name of the output sequence file and the sequence file format. 

In [10]:
# Writing FASTA files
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

sequence = 'MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAHEVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFG'
 
seq = Seq(sequence)
protein = [SeqRecord(seq, id="THEID", description='a description'),]

with open( "biopython.fa", "w") as f:
    SeqIO.write(protein, f, 'fasta')

with open( "biopython.fa" ) as f:
    print(f.read())

>THEID a description
MYGKIIFVLLLSEIVSISASSTTGVAMHTSTSSSVTKSYISSQTNDTHKRDTYAATPRAH
EVSEISVRTVYPPEEETGERVQLAHHFSEPEITLIIFG



## Connecting with biological databases

Sequences can be searched and downloaded from public databases. 

In [9]:
# Read FASTA file from NCBI GenBank
from Bio import Entrez

Entrez.email = 'A.N.Other@example.com' # Always tell NCBI who you are
handle = Entrez.efetch(db="nucleotide", id="71066805", rettype="gb")
seq_record = SeqIO.read(handle, "gb")
handle.close()

print(seq_record.id, 'with', len(seq_record.features), 'features')
print(seq_record.seq)
print(seq_record.format("fasta"))

DQ091202.1 with 6 features
TTCTGGGCCTCAGTTTCCTCATTTGTATAATAACAGAATTGGAGAGTAAATTCTTAAGAGGCTTACCAGGCTGTAATTCTAAAAAAAATGCATAAATAAACTTGCCAAGGCAGATGTTTTTAGCAGCAATTCCTGAAAGAAACGGGACCAGGAGATAAGTAGAGAAAGAGTGAAGGTCTGAAATCAAACTAATAAGACAGTCCCAGACTGTCAAGGAGAGGTATGGCTGTCATCATTCAGGCCTCACCCTGCAGAACCACACCCTGGCCTTGGCCAATCTGCTCACAAGAGCAAAAAGGGCAGGACCAGGGTTGGGCATATAAGGAAGAGTAGTGCCAGCTGCTGTTTACACTCACTTCTGACACAACTGTGTTGACTAGCAACTACCCAATCAGACACCATGGTGAATCTGACTGCTGCTGAGAAGACACAAGTCACCAACCTGTGGGGCAAGGTGAATGTGAAAGAGCTTGGTGGTGAGGCCCTGAGCAGGTTTGTATCTAGGTTGCAAGGTAGACTTAAGGAGGGTTGAGTGGGGCTGGGCATGTGGAGACAGAACAGTCTCCCAGTTTCTGACAGGCACTGACTTCCTCTGCACCSTGTGGTGCTTTCACCTTCAGGCTGCTGGTGGTCTACCCATGGACCCGGAGGTTCTTTGAACACTTTGGGGACCTGTCCACTGCTGACGCTGTCCTGCACAACGCTAAAGTGCTGGCCCATGGCGAGAAAGTGTTGACCTCCTTTGGTGAGGGCCTGAAGCACCTGGACAACCTCAAGGGCACCTTTGCCGATCTGAGCGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAATTTCAGGGTGAGTCTAGGAGACACTCTATTTTTTCTTTTCACTTTGTAGTCTTTCACTGTGATTATTTTGCTTATTTGAATTTCCTCTGTATCTCTTTTTACTCGACTATGTTTCATCATTTAGTGTTTTTTCAACT

In [10]:
# Read SWISSPROT record
from Bio import ExPASy

handle = ExPASy.get_sprot_raw('HBB_HUMAN')
prot_record = SeqIO.read(handle, "swiss")
handle.close()

print(prot_record.description)
print(prot_record.seq)

AssertionError: 

## Exercise 2.3.1

-  a FASTA file named `data/sample.fa` using BioPython and answer the following questions:
  - How many sequences are in the file?
  - What are the IDs and the lengths of the longest and the shortest sequences?
  - Select sequences longer than 500bp. What is the average length of these sequences?
  - Calculate and print the percentage of GC in each of the sequences.
  - Write the newly created sequences into a FASTA file named `long_sequences.fa` 

## Retrieving more biological data (live coding session)

## Next session

Go to our next notebook: [Session 2.4: Data project report in Jupyter](24_python_data.ipynb)